<a href="https://colab.research.google.com/github/litkm/WJBrennan-Voting/blob/main/WJBrennan_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

dataset = 'https://raw.githubusercontent.com/litkm/WJBrennan-Voting/main/WJBrennan_voting.csv'
dataset = pd.read_csv(dataset)

print(dataset.head())
print(dataset.describe())

   term  petitioner  respondent  ...   issue  issueArea      vote
0  1956         113         102  ...  100130         10  majority
1  1956           1         102  ...   90310          9  majority
2  1956         184          27  ...   80010          8  majority
3  1956         138         369  ...  120020         12  majority
4  1956         195           3  ...   40070          4  majority

[5 rows x 10 columns]
              term   petitioner  ...          issue    issueArea
count  4746.000000  4746.000000  ...    4746.000000  4746.000000
mean   1972.634640   142.193426  ...   51929.660767     5.180152
std       9.588592   106.911454  ...   35963.921687     3.598647
min    1956.000000     1.000000  ...       0.000000     0.000000
25%    1964.000000    28.000000  ...   20060.000000     2.000000
50%    1973.000000   126.000000  ...   40045.000000     4.000000
75%    1981.000000   215.000000  ...   80220.000000     8.000000
max    1989.000000   501.000000  ...  130020.000000    13.000